<center>
<img src="../../img/ods_stickers.jpg">
## Открытый курс по машинному обучению
<center>
Автор материала: Юрий Кашницкий, программист-исследователь Mail.Ru Group <br> 

Материал распространяется на условиях лицензии [Creative Commons CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/). Можно использовать в любых целях (редактировать, поправлять и брать за основу), кроме коммерческих, но с обязательным упоминанием автора материала.

# <center>Домашнее задание 6 (демо). Линейная регрессия, Lasso и RF-регрессия в задаче по определению качества вина</center>
<img src='../../img/wine_quality.jpg' width=30%>

**Заполните пропущенный код и ответьте на вопросы в [онлайн-форме](https://docs.google.com/forms/d/1gsNxgkd0VqidZp4lh9mnCQnJw3b0IFR1C4WBES86J40).**

In [1]:
# отключим всякие предупреждения Anaconda
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from sklearn.metrics.regression import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.linear_model import LinearRegression, LassoCV, Lasso, LogisticRegression
from sklearn.ensemble import RandomForestRegressor


**Будем работать с набором данных по качеству белого вина (репозиторий UCI).**
**Загружаем данные.**

In [2]:
data = pd.read_csv('winequality-white.csv', sep=';')

In [3]:
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4898 entries, 0 to 4897
Data columns (total 12 columns):
fixed acidity           4898 non-null float64
volatile acidity        4898 non-null float64
citric acid             4898 non-null float64
residual sugar          4898 non-null float64
chlorides               4898 non-null float64
free sulfur dioxide     4898 non-null float64
total sulfur dioxide    4898 non-null float64
density                 4898 non-null float64
pH                      4898 non-null float64
sulphates               4898 non-null float64
alcohol                 4898 non-null float64
quality                 4898 non-null int64
dtypes: float64(11), int64(1)
memory usage: 459.3 KB


**Отделите целевой признак, разделите обучающую выборку в отношении 7:3 (30% - под оставленную выборку, пусть random_state=17) и отмасштабируйте данные с помощью StandardScaler.**

In [40]:
# Ваш код здесь
y = data['quality'].values
x = data.iloc[:,:11].values
X_train, X_holdout, y_train, y_holdout = train_test_split(x, y, 
                                         test_size=0.3, random_state=17)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train) 
X_holdout_scaled = scaler.transform(X_holdout) 

import matplotlib
quality = data['quality'].groupby(data['quality']).count()
quality
quality.plot(kind='bar');

## Линейная регрессия

**Обучите простую линейную регрессию.**

In [39]:
linreg = LogisticRegression()
linreg.fit(X_train_scaled ,y_train)
predict_X_train = linreg.predict(X_train_scaled)
predict_X_holdout = linreg.predict(X_holdout_scaled)

**<font color='red'>Вопрос 1:</font> Каковы среднеквадратичные ошибки линейной регрессии на обучающей и отложенной выборках?**

In [38]:
print("Mean squared error (train): %.3f" % 
     mean_squared_error(y_train,  linreg.predict(X_train_scaled)))
print("Mean squared error (test): %.3f" % 
      mean_squared_error(y_holdout,linreg.predict(X_holdout_scaled)))

Mean squared error (train): 0.645
Mean squared error (test): 0.656


**Посмотрите на коэффициенты модели и отранжируйте признаки по влиянию на качество вина (учтите, что большие по модулю отрицательные значения коэффициентов тоже говорят о сильном влиянии). Создайте для этого новый небольшой DataFrame.**<br>
**<font color='red'>Вопрос 2:</font> Какой признак линейная регрессия считает наиболее сильно влияющим на качество вина?**

In [9]:
independent_columns_names = list(data.columns[:11])
coef= pd.DataFrame(linreg.coef_, columns=independent_columns_names).abs()
coef.describe()
print('density')
coef.describe().loc['mean',:].sort_values()

citric acid             0.052953
sulphates               0.109123
total sulfur dioxide    0.116773
chlorides               0.176688
fixed acidity           0.248873
free sulfur dioxide     0.253220
pH                      0.284323
volatile acidity        0.327281
alcohol                 0.330574
residual sugar          0.672867
density                 0.810330
Name: mean, dtype: float64

## Lasso-регрессия

**Обучите Lasso-регрессию с небольшим коэффициентом $\alpha = 0.01$ (слабая регуляризация). Пусть опять random_state=17.**

In [10]:
lasso1 = Lasso(alpha=0.01, random_state=17)
lasso1.fit(X_train_scaled, y_train)

Lasso(alpha=0.01, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=17,
      selection='cyclic', tol=0.0001, warm_start=False)

**Посмотрите на коэффициенты модели и отранжируйте признаки по влиянию на качество вина. Какой признак "отвалился" первым, то есть наименее важен для объяснения целевого признака в модели Lasso?**

In [11]:
lasso1.coef_

array([-0.        , -0.18847922, -0.        ,  0.25636281, -0.00274732,
        0.04308762, -0.        , -0.23549185,  0.06727655,  0.02972152,
        0.32242505])

In [12]:
#lasso1_coef = pd.DataFrame # Ваш код здесь
#lasso1_coef.sort_values # Ваш код здесь
lasso1_coef = pd.DataFrame({'coef': np.abs(lasso1.coef_)}, 
                           index=independent_columns_names)

lasso1_coef.sort_values('coef')

,coef
fixed acidity,0.000000
citric acid,0.000000
total sulfur dioxide,0.000000
chlorides,0.002747
sulphates,0.029722
free sulfur dioxide,0.043088
pH,0.067277
volatile acidity,0.188479
density,0.235492
residual sugar,0.256363


In [13]:
print("fixed acidity")

fixed acidity


**Теперь определите лучшее значение $\alpha$ в процессе кросс-валидации 5-кратной кросс-валидации. Используйте LassoCV и random_state=17.**

In [14]:
alphas = np.logspace(-6, 2, 200)
lasso_cv = LassoCV(alphas=alphas, cv=5, random_state=17)
lasso_cv.fit(X_train_scaled, y_train)

LassoCV(alphas=array([1.00000000e-06, 1.09698580e-06, 1.20337784e-06, 1.32008840e-06,
       1.44811823e-06, 1.58856513e-06, 1.74263339e-06, 1.91164408e-06,
       2.09704640e-06, 2.30043012e-06, 2.52353917e-06, 2.76828663e-06,
       3.03677112e-06, 3.33129479e-06, 3.65438307e-06, 4.00880633e-06,
       4.39760361e-06, 4.82410870e-06, 5.29197874e-06, 5.80522552e-06,
       6.36824994e-06, 6.98587975e-0...
       3.61234270e+01, 3.96268864e+01, 4.34701316e+01, 4.76861170e+01,
       5.23109931e+01, 5.73844165e+01, 6.29498899e+01, 6.90551352e+01,
       7.57525026e+01, 8.30994195e+01, 9.11588830e+01, 1.00000000e+02]),
        copy_X=True, cv=5, eps=0.001, fit_intercept=True, max_iter=1000,
        n_alphas=100, n_jobs=None, normalize=False, positive=False,
        precompute='auto', random_state=17, selection='cyclic', tol=0.0001,
        verbose=False)

In [15]:
lasso_cv.alpha_

0.0002833096101839324

**Выведите коэффициенты "лучшего" Lasso в порядке убывания влияния на качество вина. **<br>
**<font color='red'>Вопрос 3:</font> Какой признак "обнулился первым" в настроенной модели LASSO?**

In [16]:
lasso_cv_coef = pd.DataFrame # Ваш код здесь
lasso_cv_coef.sort_values # Ваш код здесь
lasso1_coef = pd.DataFrame({'coef': np.abs(lasso_cv.coef_)}, 
                           index=independent_columns_names)

lasso1_coef.sort_values('coef'), print('citric acid')

citric acid


(                          coef
 citric acid           0.000000
 chlorides             0.006933
 total sulfur dioxide  0.012969
 free sulfur dioxide   0.042698
 sulphates             0.060939
 fixed acidity         0.093295
 alcohol               0.137115
 pH                    0.146549
 volatile acidity      0.192049
 residual sugar        0.526883
 density               0.648161, None)

**Оцените среднеквадратичную ошибку модели на обучающей и тестовой выборках.**<br>
**<font color='red'>Вопрос 4:</font> Каковы среднеквадратичные ошибки настроенной LASSO-регрессии на обучающей и отложенной выборках?**

In [17]:
print("Mean squared error (train): %.3f" %
           mean_squared_error(y_train,  lasso_cv.predict(X_train_scaled)))
print("Mean squared error (test): %.3f" % 
      mean_squared_error(y_holdout, lasso_cv.predict(X_holdout_scaled)))

Mean squared error (train): 0.558
Mean squared error (test): 0.583


## Случайный лес

**Обучите случайный лес с параметрами "из коробки", фиксируя только random_state=17.**

In [18]:
forest = RandomForestRegressor(random_state=17)
forest.fit(X_train_scaled, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=10,
                      n_jobs=None, oob_score=False, random_state=17, verbose=0,
                      warm_start=False)

**<font color='red'>Вопрос 5:</font> Каковы среднеквадратичные ошибки случайного леса на обучающей выборке, на кросс-валидации (cross_val_score с scoring='neg_mean_squared_error' и остальными параметрами по умолчанию) и на отложенной выборке?**

In [19]:
print("Mean squared error (train): %.3f" % 
      mean_squared_error(y_train,  forest.predict(X_train_scaled)))
print("Mean squared error (cv): %.3f" % 
      abs(np.mean(cross_val_score(forest, X_train_scaled, y_train,
                                  scoring='neg_mean_squared_error'))))
print("Mean squared error (test): %.3f" % 
      mean_squared_error(y_holdout, forest.predict(X_holdout_scaled)))


Mean squared error (train): 0.075
Mean squared error (cv): 0.460
Mean squared error (test): 0.422


**Настройте параметры min_samples_leaf и max_depth с помощью GridSearchCV и опять проверьте качество модели на кросс-валидации и на отложенной выборке.**

In [20]:
forest_params = {'max_depth': list(range(10, 25)), 
                 'min_samples_leaf': list(range(1, 8)),
                 'max_features': list(range(6,12))}

locally_best_forest = GridSearchCV(forest,forest_params)
locally_best_forest.fit(X_train_scaled, y_train)

GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=RandomForestRegressor(bootstrap=True, criterion='mse',
                                             max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators=10, n_jobs=None,
                                             oob_score=False, random_state=17,
                                             verbose=0, warm_start=False),
             iid='warn', n_jobs=None,
             param_grid={'max_depth': [10, 11, 12, 13, 14

In [21]:
locally_best_forest.best_params_, locally_best_forest.best_score_

({'max_depth': 22, 'max_features': 7, 'min_samples_leaf': 1},
 0.4240630944354115)

**К сожалению, результаты  GridSearchCV не полностью воспроизводимы (могут отличаться на разных платформах даже при фиксировании *random_state*). Поэтому обучите лес с параметрами max_depth=19, max_features=7, и min_samples_leaf=1 (лучшие в моем случае).**<br>
**<font color='red'>Вопрос 6:</font> Каковы среднеквадратичные ошибки настроенного случайного леса на обучающей выборке, на кросс-валидации (cross_val_score с scoring='neg_mean_squared_error') и на отложенной выборке?**

In [28]:
forest_params = {'max_depth': [19], 
                 'min_samples_leaf': [1],
                 'max_features': [7]}

locally_best_forest = GridSearchCV(forest, forest_params)
locally_best_forest.fit(X_train_scaled, y_train)

GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=RandomForestRegressor(bootstrap=True, criterion='mse',
                                             max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators=10, n_jobs=None,
                                             oob_score=False, random_state=17,
                                             verbose=0, warm_start=False),
             iid='warn', n_jobs=None,
             param_grid={'max_depth': [19], 'max_features

In [29]:
locally_best_forest.best_estimator_, locally_best_forest.best_params_

(RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=19,
                       max_features=7, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=17, verbose=0,
                       warm_start=False),
 {'max_depth': 19, 'max_features': 7, 'min_samples_leaf': 1})

In [31]:
print("Mean squared error (cv): %.3f" %  
abs(np.mean(cross_val_score(locally_best_forest.best_estimator_, 
     X_train_scaled, y_train, scoring='neg_mean_squared_error'))))
print("Mean squared error (test): %.3f" % 
      mean_squared_error(y_holdout, 
      locally_best_forest.predict(X_holdout_scaled)))   


Mean squared error (cv): 0.457
Mean squared error (test): 0.410


**Оцените важность признаков с помощью случайного леса.**<br>
**<font color='red'>Вопрос 7:</font> Какой признак оказался главным в настроенной модели случайного леса?**

In [33]:
rf_importance = pd.DataFrame(locally_best_forest.best_estimator_.feature_importances_, 
                             columns=['coef'], index=independent_columns_names)
print('alcohol')
rf_importance.sort_values('coef')

alcohol


,coef
sulphates,0.061184
citric acid,0.062945
fixed acidity,0.064268
chlorides,0.067982
density,0.069367
residual sugar,0.070160
total sulfur dioxide,0.071318
pH,0.072806
free sulfur dioxide,0.116147
volatile acidity,0.119393


**Сделайте выводы о качестве моделей и оценках влияния признаков на качество вина с помощью этих трех моделей.**

In [ ]:
В каждой модели свои оценки влияния и они не совпадают. 
При вычислении среднеквадратичных ошибок лучше себя показал случайный лесб а хуже - линейная регрессия и лассо.
Потому что качество вина не зависит линейно от особенностей - случайный лес работает лучше.
